In [1]:
from pyspark.sql import SparkSession        
# Create a SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQLExampleApp")
  .getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1700874866306_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
csv_file = "s3://jupyterjorge/raw/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-100K.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

# PySpark

In [5]:
df.groupBy("Nombre departamento").count().orderBy("count", ascending=False).show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Nombre departamento|count|
+-------------------+-----+
|             BOGOTA|30016|
|       BARRANQUILLA|13065|
|          ATLANTICO|10994|
|              VALLE|10404|
|          CARTAGENA| 8333|
|          ANTIOQUIA| 4554|
|             NARIÑO| 3520|
|       CUNDINAMARCA| 2827|
|           AMAZONAS| 2317|
|              CHOCO| 1636|
+-------------------+-----+
only showing top 10 rows

In [6]:
df.groupBy("Nombre municipio").count().orderBy("count", ascending=False).show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|Nombre municipio|count|
+----------------+-----+
|          BOGOTA|30016|
|    BARRANQUILLA|13065|
|       CARTAGENA| 8333|
|            CALI| 7747|
|         SOLEDAD| 6233|
|         LETICIA| 2194|
|        MEDELLIN| 2137|
|          TUMACO| 1501|
|    BUENAVENTURA| 1453|
|          QUIBDO| 1367|
+----------------+-----+
only showing top 10 rows

In [7]:
df.groupBy("Fecha de diagnóstico").count().orderBy(col("count").desc(), "Fecha de diagnóstico").show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Fecha de diagn?stico` cannot be resolved. Did you mean one of the following? [`Fecha de diagnóstico`, `Fecha de muerte`, `Fecha de notificación`, `Fecha de recuperación`, `ID de caso`].;
'Aggregate ['Fecha de diagn?stico], ['Fecha de diagn?stico, count(1) AS count#212L]
+- Relation [fecha reporte web#17,ID de caso#18,Fecha de notificación#19,Código DIVIPOLA departamento#20,Nombre departamento#21,Código DIVIPOLA municipio#22,Nombre municipio#23,Edad#24,Unidad de medida de edad#25,Sexo#26,Tipo de contagio#27,Ubicación del caso#28,Estado#29,Código ISO del país#30,Nombre del país#31,Recuperado#32,Fecha de inicio de síntomas#33,Fecha de muerte#34,Fecha de diagnóstico#35,Fecha de recuperación#36,Tipo de recuperación#37,Pertenencia étnica#38,Nombre del grupo étnico#39] csv

Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1700874866306_0004/container

In [8]:
df.groupBy("Edad").count().orderBy("Edad").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Edad|count|
+----+-----+
|   1|  485|
|   2|  440|
|   3|  449|
|   4|  373|
|   5|  425|
|   6|  431|
|   7|  442|
|   8|  461|
|   9|  467|
|  10|  530|
|  11|  566|
|  12|  562|
|  13|  531|
|  14|  580|
|  15|  560|
|  16|  600|
|  17|  685|
|  18| 1160|
|  19| 1567|
|  20| 1674|
+----+-----+
only showing top 20 rows

In [17]:
total = df.count()
from pyspark.sql.functions import col

recovered = df.filter(col("Recuperado") == "Recuperado").count()
deceased = df.filter(col("Recuperado") == "Fallecido").count() + df.filter(col("Recuperado") == "fallecido").count()

print(f"La tasa de recuperación es: {(recovered / total) * 100:.2f}% y de mortalidad es de {(deceased / total) * 100:.2f}%")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La tasa de recuperaci?n es: 94.83% y de mortalidad es de 4.66%

In [10]:
df.select('Recuperado').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|Recuperado|
+----------+
|Recuperado|
| fallecido|
| Fallecido|
|       N/A|
+----------+

# Ahora SparkSQL

In [19]:
df.createOrReplaceTempView("covid_data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
spark.sql("""
    SELECT `Nombre departamento`, COUNT(*) as total_cases
    FROM covid_data
    GROUP BY `Nombre departamento`
    ORDER BY total_cases DESC
    LIMIT 10
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+
|Nombre departamento|total_cases|
+-------------------+-----------+
|             BOGOTA|      30016|
|       BARRANQUILLA|      13065|
|          ATLANTICO|      10994|
|              VALLE|      10404|
|          CARTAGENA|       8333|
|          ANTIOQUIA|       4554|
|             NARIÑO|       3520|
|       CUNDINAMARCA|       2827|
|           AMAZONAS|       2317|
|              CHOCO|       1636|
+-------------------+-----------+

In [21]:
spark.sql("""
    SELECT `Nombre municipio`, COUNT(*) as total_cases
    FROM covid_data
    GROUP BY `Nombre municipio`
    ORDER BY total_cases DESC
    LIMIT 10
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|Nombre municipio|total_cases|
+----------------+-----------+
|          BOGOTA|      30016|
|    BARRANQUILLA|      13065|
|       CARTAGENA|       8333|
|            CALI|       7747|
|         SOLEDAD|       6233|
|         LETICIA|       2194|
|        MEDELLIN|       2137|
|          TUMACO|       1501|
|    BUENAVENTURA|       1453|
|          QUIBDO|       1367|
+----------------+-----------+

In [22]:
spark.sql("""
    SELECT `Fecha de diagnóstico`, COUNT(*) as total_cases
    FROM covid_data
    GROUP BY `Fecha de diagnóstico`
    ORDER BY total_cases DESC, `Fecha de diagnóstico`
    LIMIT 10
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Fecha de diagn?stico` cannot be resolved. Did you mean one of the following? [`covid_data`.`Fecha de diagnóstico`, `covid_data`.`Estado`, `covid_data`.`Sexo`, `covid_data`.`Edad`, `covid_data`.`ID de caso`].; line 2 pos 11;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Sort ['total_cases DESC NULLS LAST, 'Fecha de diagn?stico ASC NULLS FIRST], true
      +- 'Aggregate ['Fecha de diagn?stico], ['Fecha de diagn?stico, count(1) AS total_cases#540L]
         +- SubqueryAlias covid_data
            +- View (`covid_data`, [fecha reporte web#17,ID de caso#18,Fecha de notificación#19,Código DIVIPOLA departamento#20,Nombre departamento#21,Código DIVIPOLA municipio#22,Nombre municipio#23,Edad#24,Unidad de medida de edad#25,Sexo#26,Tipo de contagio#27,Ubicación del caso#28,Estado#29,Código ISO del país#30,Nombre del país#31,Recuperado#32,Fecha de inicio de síntomas#33,Fecha de muerte#34,Fecha de dia

In [23]:
spark.sql("""
    SELECT Edad, COUNT(*) as total_cases
    FROM covid_data
    GROUP BY Edad
    ORDER BY Edad
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+
|Edad|total_cases|
+----+-----------+
|   1|        485|
|   2|        440|
|   3|        449|
|   4|        373|
|   5|        425|
|   6|        431|
|   7|        442|
|   8|        461|
|   9|        467|
|  10|        530|
|  11|        566|
|  12|        562|
|  13|        531|
|  14|        580|
|  15|        560|
|  16|        600|
|  17|        685|
|  18|       1160|
|  19|       1567|
|  20|       1674|
+----+-----------+
only showing top 20 rows

In [24]:
spark.sql("""
    SELECT
        (SUM(CASE WHEN Recuperado = 'Recuperado' THEN 1 ELSE 0 END) / COUNT(*)) * 100 as recovery_rate
    FROM covid_data
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|    recovery_rate|
+-----------------+
|94.83200000000001|
+-----------------+

In [25]:
mortality_rate_result = spark.sql( """
    SELECT 
        (SUM(CASE WHEN `Fecha de muerte` IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*)) * 100 as mortality_rate
    FROM covid_data
"""
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|mortality_rate|
+--------------+
|         5.633|
+--------------+